# Annotation tool for time series data

By: Stefania Russo, Kris Villez
Copyright: 2018, distributed with BSD3 license 

## The challenge

In the context of the ADASen project, we want to address research questions regarding the utility of supervised and unsupervised machine learning models in anomaly detection for environmental systems. We have therefore selected a range of anomaly detection methods for benchmarking on data sets produced by six infrastructures at Eawag.

Critical to the benchmarking is the availability of fully labelled training and test data sets of normal and abnormal behavior in environmental data. 
An annotation tool has therefore being developed to perform the labelling procedure.

This notebook shows an application of the labelling procedure to time series data. Here, each time series is a univariate 24h signal from a spatially-distributed, low-power sensor network.

Each series is visualised as a 3am-3am time series.

## Current method

Below are described the steps for data access, data preparation, visualization and labelling procedure.

- The data is in the form of .csv data files. Each data file consists of many 24h sets across 3 sensors.
- Corruption checks are performed and dates cointaining corrupted time-series are removed
- The labelling procedure starts and the first plots are displayed. The 3 plots at the top are univariate sensor signals, where the bottom plot shows a collection of these signals.  

- The annotation tool allows the labelling expert to interactivelly select multiple portions of the time series by moving through the data with the mouse cursor.

- Each time the button 'Next' is clicked, all the selected areas (time index and sensor value) are saved together with information about the date stamp date. At the end of the procedure, the user can easily access to the anomaly labels in an easy manner.

- When the process is over, the plots need to be closed and then the cell 'Save labelled data' has to be run 

- Note: if the user wants to change any of his selections, he needs to move forward to the next plot by clicking 'Next', perform a selection of the anomalous data, and then go back and restart.


# Usage

 - Install python and open this Jupyter notebook 
 - Paste your working directory into path_all

# Iniziatization

In [ ]:
# Import Statements
%run functions/startup.py
random.seed(4)

### Options

In [ ]:
path_base = str(Path.cwd())
folder = '/data_examples/UWO_data/'  # Select data folder

path_all = path_base + folder
save_path = path_all     # Destination folder to for labelled data
name_of_file1 = 's1.csv'

name_of_file_l1 = "Labels_raw_"
name_of_file_l1_time = "Labels_Anomaly_"

In [ ]:
# FUNCTIONS

# ADD zeros and ones with dictionary mapping
mapper_dict = {'left_only': 0, 'both': 1}

def mp(entry):
    """
    maps new values
    """
    return mapper_dict[entry] if entry in mapper_dict else entry
mp = np.vectorize(mp)

# Load data and basic sanity checks

In [ ]:
# Load data

completePath = os.path.join(path_all, name_of_file1) 
df = pd.read_csv(completePath)

df2 = df.copy(deep=True)
df2['date'], df2['time'] = df2['time'].str.split(' ', 1).str

sr0 = df2.keys()[1]
sr1 = df2.keys()[2]
sr2 = df2.keys()[3]
print('Sensor names:',sr0,',', sr1,',', sr2)

# Create datetime 

df2['date'] = [x.date() for x in (pd.to_datetime([i for i in df2['date']], format='%Y-%m-%d'))] 
df2['time'] = [x.time() for x in (pd.to_datetime([i for i in df2['time']], format='%H:%M:%S'))]   # remove primes from the time
df1 = df2.copy(deep=True)
df2.set_index(['date','time'], inplace=True)

df_bf_00 = df2[sr0]
df_bf_01 = df2[sr1]
df_bf_02 = df2[sr2]

## Basic sanity checks

In [ ]:
# Accessing dates
i_date = df2.index.get_level_values(0)                                      # get all dates
idx_date = np.unique(df2.index.get_level_values(0), return_index=True)[1]      # get index of unique dates
date_list = i_date[idx_date]   # get list of all dates
print('Unique dates:',date_list)

for pl_i in range(len(date_list)):
    if len(df_bf_00[date_list[pl_i]].values) != 288:
        print('Corrupted date:', date_list[pl_i])
        print('Corrupted date index:',pl_i)
        print('Corrupted date shape:', df2.loc[date_list[pl_i]].shape)  

# Removing dates where more than 1/3 of data is missing

data_df2 = df2.copy()
for pl_i in range(len(date_list)):
    if len(df_bf_00[date_list[pl_i]].values) < 192:
        data_df2.drop(date_list[pl_i],level='date',inplace=True)

# Compute again date index
# Accessing dates
i_date = data_df2.index.get_level_values(0)                                      # get all dates
idx_date = np.unique(data_df2.index.get_level_values(0), return_index=True)[1]      # get index of unique dates
date_list = i_date[idx_date]   # get list of all dates

df_bf_00 = data_df2[sr0]
df_bf_01 = data_df2[sr1]
df_bf_02 = data_df2[sr2]

In [ ]:
# Dates and times
data_time = []
for pl_i in idx_date:                             # create data_time indeces to have access later
    time = data_df2.loc[i_date[pl_i]].index
    data_time.append(time)                        # associated to every date segment

In [ ]:
mm = []
for i in range(len(data_time)):
    minutes = []
    for j in data_time[i]:
        mins = (j.hour * 60 + j.minute)
        minutes.append(mins) 
    mm.append(minutes)
    
time_int = mm.copy()

# Plotting

In [ ]:
%matplotlib tk

data1 = []
data2 = []
data3 = []
data123 = []

itera = date_list

import numpy as np
import matplotlib.pylab as pl
import matplotlib.gridspec as gridspec

gs = gridspec.GridSpec(2, 3)

fig = plt.figure()
#plt.axis([0, 24, -3, 100])

ax1 = fig.add_subplot(gs[0, 0]) # row 0, col 0
ax2 = fig.add_subplot(gs[0, 1]) # row 0, col 1
ax3 = fig.add_subplot(gs[0, 2]) # row 0, col 1
ax4 = fig.add_subplot(gs[1, :]) # row 1, span all columns

ax1.set_title(sr0, fontdict=None, pad=None)
ax2.set_title(sr1, fontdict=None, pad=None)
ax3.set_title(sr2, fontdict=None, pad=None)
full = sr0 + ' '+ sr1 +''+ sr2
ax4.set_title(full, fontdict=None, pad=None)

fig.suptitle(str(date_list[0]), fontsize=12)

ax1.set_ylim([-3,100])
ax2.set_ylim([-3,100])
ax3.set_ylim([-3,100])
ax4.set_ylim([-3,100])

for pl_i in range(len(date_list)): 
    ax1.plot(time_int[pl_i], df_bf_00[date_list[pl_i]].values, '#C0C0C0', lw=2)
    ax2.plot(time_int[pl_i], df_bf_01[date_list[pl_i]].values, '#C0C0C0', lw=2)
    ax3.plot(time_int[pl_i], df_bf_02[date_list[pl_i]].values,  '#C0C0C0',lw=2) 
    
l, = ax1.plot(time_int[0], df_bf_00[date_list[0]].values, '#1E90FF', lw=2)     #the first one is the one in blue
l2, = ax2.plot(time_int[0], df_bf_01[date_list[0]].values, '#8B008B')
l3, = ax3.plot(time_int[0], df_bf_02[date_list[0]].values,'#FFDAB9')


ll1, = ax4.plot(time_int[0], df_bf_00[date_list[0]].values, '#1E90FF')
ll2, = ax4.plot(time_int[0], df_bf_01[date_list[0]].values, '#8B008B')
ll3, = ax4.plot(time_int[0],  df_bf_02[date_list[0]].values, '#FFDAB9')




############### Buttons widget  ####################

class Index(object):
    ind = 0

    def next(self, event):
        self.ind += 1
        i = self.ind % len(itera)

        #ydata0 will be the plot alone
        ydata1 = df_bf_00[date_list[i]].values   
        ydata2 = df_bf_01[date_list[i]].values 
        ydata3 = df_bf_02[date_list[i]].values
        xdata = time_int[i]          
        
        l.set_ydata(ydata1)
        l.set_xdata(xdata)
        l2.set_ydata(ydata2)
        l2.set_xdata(xdata)
        l3.set_ydata(ydata3)
        l3.set_xdata(xdata)
        
        ll1.set_ydata(ydata1)
        ll2.set_ydata(ydata2)
        ll3.set_ydata(ydata3) 
        
        ll1.set_xdata(xdata) 
        ll2.set_xdata(xdata)
        ll3.set_xdata(xdata)
        
        if (i == (0)):
            fig.suptitle('End of data files - restarting with data file ' + str(date_list[i]), fontsize=12)
        else: 
            fig.suptitle(str(date_list[i]), fontsize=12)
            
        plt.draw()

    def prev(self, event):
        self.ind -= 1
        i = self.ind % len(itera)
        
        #ydata0 will be the plot alone
        ydata1 = df_bf_00[date_list[i]].values 
        ydata2 = df_bf_01[date_list[i]].values 
        ydata3 = df_bf_02[date_list[i]].values
        xdata = time_int[i]  
        
        l.set_ydata(ydata1)
        l.set_xdata(xdata)
        
        l2.set_ydata(ydata2)
        l2.set_xdata(xdata)
        
        l3.set_ydata(ydata3)
        l3.set_xdata(xdata)

        ll1.set_ydata(ydata1)
        ll2.set_ydata(ydata2)
        ll3.set_ydata(ydata3) 

        ll1.set_xdata(xdata) 
        ll2.set_xdata(xdata)
        ll3.set_xdata(xdata)
        

        if (i == (0)):
            fig.suptitle('End of data files - restarting with data file ' + str(date_list[i]), fontsize=12)
        else: 
            fig.suptitle(str(date_list[i]), fontsize=12)
            
        plt.draw()

callback = Index()

axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.81, 0.05, 0.1, 0.075])
bnext = Button(axnext, 'Next')
bnext.on_clicked(callback.next)

bprev = Button(axprev, 'Previous')
bprev.on_clicked(callback.prev)

"""
valore = '11'
def presskey(event):
    print('Pressed key = ', event.key)
    #sys.stdout.flush()    
    global valore 
    valore = event.key       
    return valore
"""

def onselect1(xmin, xmax):
    x = time_int[callback.ind % len(itera)]
    y = df_bf_00[date_list[callback.ind % len(itera)]].values 
    today = date_list[callback.ind % len(itera)]
   
    indmin1, indmax1 = np.searchsorted(x, (xmin, xmax))
    indmax1 = min(len(x) - 1, indmax1)
    thisx = x[indmin1:indmax1]
    thisy = y[indmin1:indmax1]    
    nplist = np.array([today.date() for i in range(len(thisx))])
        
    a1 = np.c_[nplist, thisx, thisy]
    global data1
    data1.extend(a1)
    #np.savetxt(completeName_label_1, data1)

        

def onselect2(xmin, xmax):
    x = time_int[callback.ind % len(itera)]
    y = df_bf_01[date_list[callback.ind % len(itera)]].values 
    today = date_list[callback.ind % len(itera)]
    
    indmin, indmax = np.searchsorted(x, (xmin, xmax))
    indmax = min(len(x) - 1, indmax)
    thisx = x[indmin:indmax]
    thisy = y[indmin:indmax]
    nplist = np.array([today.date() for i in range(len(thisx))])
    
    a2 = np.c_[nplist, thisx, thisy]
    global data2
    data2.extend(a2)
    

def onselect3(xmin, xmax):
    x = time_int[callback.ind % len(itera)]
    y = df_bf_02[date_list[callback.ind % len(itera)]].values 
    today = date_list[callback.ind % len(itera)]
    
    indmin, indmax = np.searchsorted(x, (xmin, xmax))
    indmax = min(len(x) - 1, indmax)
    thisx = x[indmin:indmax]
    thisy = y[indmin:indmax]
    nplist = np.array([today.date() for i in range(len(thisx))])
    
    a3 = np.c_[nplist, thisx, thisy]
    global data3
    data3.extend(a3)

def onselect4(xmin, xmax):
    x = time_int[callback.ind % len(itera)]
    y1 = df_bf_00[date_list[callback.ind % len(itera)]].values 
    y2 = df_bf_01[date_list[callback.ind % len(itera)]].values
    y3 = df_bf_02[date_list[callback.ind % len(itera)]].values
    today = date_list[callback.ind % len(itera)]
    
    indmin, indmax = np.searchsorted(x, (xmin, xmax))
    indmax = min(len(x) - 1, indmax)
    
    thisx = x[indmin:indmax]
    thisy1 = y1[indmin:indmax]
    thisy2 = y2[indmin:indmax]
    thisy3 = y3[indmin:indmax] 
    nplist = np.array([today.date() for i in range(len(thisx))])
        
    # save
    a123 = np.c_[nplist, thisx, thisy1, thisy2, thisy3]
    global data123
    data123.extend(a123)

    
"""
# Connect key event to figure
fig.canvas.mpl_connect('key_press_event',presskey)
"""

#class1 = Onselect_1()

spans1 = SpanSelector(ax1, onselect1, 'horizontal', useblit=False,
                      rectprops=dict(alpha=0.5, facecolor='red'), span_stays=True)
span2 = SpanSelector(ax2, onselect2, 'horizontal', useblit=True,
                    rectprops=dict(alpha=0.5, facecolor='red'), span_stays=True )
span3 = SpanSelector(ax3, onselect3, 'horizontal', useblit=True,
                    rectprops=dict(alpha=0.5, facecolor='red'), span_stays=False)
span4 = SpanSelector(ax4, onselect4, 'horizontal', useblit=True,
                    rectprops=dict(alpha=0.5, facecolor='red') , span_stays=False)

## Save labels

In [ ]:
data1 = pd.DataFrame(data1, columns=['date','time_m', sr0])
data2 = pd.DataFrame(data2, columns=['date','time_m', sr1])
data3 = pd.DataFrame(data3, columns=['date','time_m', sr2])
data123 = pd.DataFrame(data123, columns=['date','time_m', sr0, sr1, sr2])

# Save raw labels
# data1.to_csv(os.path.join(save_path, name_of_file_l1+sr0 + ".csv") )
# data2.to_csv(os.path.join(save_path, name_of_file_l1+sr1 + ".csv") )
# data3.to_csv(os.path.join(save_path, name_of_file_l1+sr2 + ".csv") )
# data123.to_csv(os.path.join(save_path, name_of_file_l1+sr0+sr1+sr2 + ".csv") )


#### Back to time

In [ ]:
data1_l = data1.copy(deep=True)
data2_l = data2.copy(deep=True)
data3_l = data3.copy(deep=True)
data123_l = data123.copy(deep=True)

In [ ]:
data1_l["time"] = None
data2_l["time"] = None 
data3_l["time"] = None 
data123_l["time"] = None 

for iSample in range(data1_l.shape[0]):
    entry = data1.loc[iSample,'time_m']
    time = str(timedelta(minutes=int(entry)))
    data1_l.loc[iSample,'time'] = time
data1_l = data1_l[['date', 'time_m', 'time', sr0]]
data1_l.drop(['time_m'], axis=1, inplace=True)
data1_l['date'] = pd.to_datetime((data1_l['date']), format='%Y-%m-%d')
data1_l['time'] = [x.time() for x in (pd.to_datetime([i for i in data1_l['time']], format='%H:%M:%S'))] 
    

for iSample in range(data2_l.shape[0]):
    entry = data2.loc[iSample,'time_m']
    time = str(timedelta(minutes=int(entry)))
    data2_l.loc[iSample,'time'] = time
data2_l = data2_l[['date', 'time_m', 'time', sr1]]
data2_l.drop(['time_m'], axis=1, inplace=True)
data2_l['date'] = pd.to_datetime((data2_l['date']), format='%Y-%m-%d')
data2_l['time'] = [x.time() for x in (pd.to_datetime([i for i in data2_l['time']], format='%H:%M:%S'))] 
    

for iSample in range(data3_l.shape[0]):
    entry = data3.loc[iSample,'time_m']
    time = str(timedelta(minutes=int(entry)))
    data3_l.loc[iSample,'time'] = time
data3_l = data3_l[['date', 'time_m', 'time', sr2]]
data3_l.drop(['time_m'], axis=1, inplace=True)
data3_l['date'] = pd.to_datetime((data3_l['date']), format='%Y-%m-%d')
data3_l['time'] = [x.time() for x in (pd.to_datetime([i for i in data3_l['time']], format='%H:%M:%S'))] 
   

for iSample in range(data123_l.shape[0]):
    entry = data123.loc[iSample,'time_m']
    time = str(timedelta(minutes=int(entry)))
    data123_l.loc[iSample,'time'] = time
data123_l = data123_l[['date', 'time_m', 'time', sr0, sr1, sr2]]
data123_l.drop(['time_m'], axis=1, inplace=True)
data123_l['date'] = pd.to_datetime((data123_l['date']), format='%Y-%m-%d')
data123_l['time'] = [x.time() for x in (pd.to_datetime([i for i in data123_l['time']], format='%H:%M:%S'))] 

In [ ]:
data_lab_1 = pd.DataFrame(df_bf_00.copy())
data_lab_1 = data_lab_1.reset_index(level=[0,1])

data_lab_2 = pd.DataFrame(df_bf_01.copy())
data_lab_2 = data_lab_2.reset_index(level=[0,1])

data_lab_3 = pd.DataFrame(df_bf_02.copy())
data_lab_3 = data_lab_3.reset_index(level=[0,1])

data_lab_123 = pd.DataFrame(df_bf_00.copy())
data_lab_123[sr1] = pd.DataFrame(df_bf_01)
data_lab_123[sr2] = pd.DataFrame(df_bf_02)
data_lab_123 = data_lab_123.reset_index(level=[0,1])

In [ ]:
#labels_df = pd.merge(data_lab_123, data123_l, on = ['date', 'time'], how='left', indicator=True)
labels_df_1 = pd.merge(data_lab_1, data1_l, how='left', indicator=True)
labels_df_2 = pd.merge(data_lab_2, data2_l, how='left', indicator=True)
labels_df_3 = pd.merge(data_lab_3, data3_l, how='left', indicator=True)
labels_df_123 = pd.merge(data_lab_123, data123_l, how='left', indicator=True)

In [ ]:
labels_df_1 ['_merge'] = mp(labels_df_1['_merge'])
labels_df_2 ['_merge'] = mp(labels_df_2['_merge'])
labels_df_3 ['_merge'] = mp(labels_df_3['_merge'])
labels_df_123 ['_merge'] = mp(labels_df_123['_merge'])

In [ ]:
labels_df_1 = labels_df_1.rename(index=str, columns={"_merge": "Anomaly"})
labels_df_2 = labels_df_2.rename(index=str, columns={"_merge": "Anomaly"})
labels_df_3 = labels_df_3.rename(index=str, columns={"_merge": "Anomaly"})
labels_df_123 = labels_df_123.rename(index=str, columns={"_merge": "Anomaly"})


In [ ]:
labels_df_1.to_csv(os.path.join(save_path, name_of_file_l1_time+sr0 + ".csv") )
labels_df_2.to_csv(os.path.join(save_path, name_of_file_l1_time+sr1 + ".csv") )
labels_df_3.to_csv(os.path.join(save_path, name_of_file_l1_time+sr2 + ".csv") )
labels_df_123.to_csv(os.path.join(save_path, name_of_file_l1_time+sr0+sr1+sr2 + ".csv") )            
